## Protocole à suivre pour l'ensemble de la procédure

### Importation des librairies

In [ ]:
import pandas
pandas.__version__
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


### Création du fichier avec les colonnes date, numero compta et plan, débit, crédit, solde progressif

In [ ]:
# le read_excel prend en argument le chemin qui va le mener vers le fichier voulu
grand_livre = pd.read_excel("Grand livre 2020-2021.xls")
# pour vérifier qu'on a bien le fichier, afficher le dataframe 
grand_livre

In [ ]:
# on sélectionne les colonnes avec la date, le plan_compta, le débit, le crédit, le solde progressif
# à noter que des erreurs pourraient survenir si le grand livre ne suit pas le même format 
data = grand_livre[["MON BRICO ETS AH-HOT","Unnamed: 1","Unnamed: 14","Unnamed: 16","Unnamed: 18"]]
# on sélectionne les lignes qui nous intéressent, ie. celles qui ont un numéro compta ou une date
data_filtered_2 = data[data["Unnamed: 18"].notna() | data["Unnamed: 1"].notna()]
data_filtered_3 = data_filtered_2[data_filtered_2["MON BRICO ETS AH-HOT"] != "Date"]
data_filtered_4 = data_filtered_3[data_filtered_3["MON BRICO ETS AH-HOT"] != "Total Grand Livre"]
# vérification 
data_filtered_4


In [ ]:
data_filtered_5 = data_filtered_4


In [ ]:
# ce bloc permet d'attribuer à chaque date le numéro compta et de filtrer les lignes
# ignorer les erreurs
for i in data_filtered_5["Unnamed: 1"].index: 
    if pd.isna(data_filtered_4["Unnamed: 1"][i]) :
        data_filtered_5["Unnamed: 1"][i] = data_filtered_5["Unnamed: 1"][i-1]
        
data_filtered_6 = data_filtered_5[data_filtered_5["Unnamed: 18"].notna()]
data_filtered_7 = data_filtered_6[data_filtered_6["MON BRICO ETS AH-HOT"].notna()]



In [ ]:
# vérification 
data_filtered_7


### Renommer les colonnes


In [ ]:
# attention, python est sensible à la casse
data_filtered_7 = data_filtered_7.rename(columns = {'MON BRICO ETS AH-HOT' : 'date', 'Unnamed: 1': 'plan_compta', 'Unnamed: 14': 'debit', 'Unnamed: 16':'credit', 'Unnamed: 18': 'solde progressif'})
# vérification
data_filtered_7


### Split des plans comptables

In [ ]:
# supprimer le mot 'Suite' 
data_filtered_7['plan_compta'] = data_filtered_7['plan_compta'].str.replace('Suite ','')
# supprimer les chaines de caractères après le premier " " pour ne garder que le numéro compta
var_test = data_filtered_7['plan_compta'].str.split(' ').str[0]
# assigner à plan_compta la nouvelle valeur
data_filtered_7['plan_compta'] = var_test
# vérification
data_filtered_7


### Merge de deux dataframes

In [ ]:
# on va merge avec le plan comptable  
plan_comptable = pd.read_excel("Plan comptable.xls")


In [ ]:
# renommer la colonne 'N° du compte' pour pouvoir faire le lien des deux dataframes
plan_comptable = plan_comptable.rename(columns = {'N° du compte' : 'plan_compta'})

In [ ]:
#afficher le plan compable
plan_comptable


In [ ]:
grand_livre_V1 = data_filtered_7.merge(plan_comptable, how='left', on='plan_compta')
# vérification 
grand_livre_V1


In [ ]:
# sélection des colonnes qui nous intéressent
grand_livre_V2 = grand_livre_V1[["date","plan_compta","Intitulé du compte général","debit","credit","solde progressif"]]
# vérification 
grand_livre_V2


### Création d'une colonne avec le premier chiffre du numéro compta

In [ ]:
# ne pas faire attention aux erreurs
grand_livre_V2["categorie"] = grand_livre_V2["plan_compta"]

for i in range (grand_livre_V2.shape[0]):
    grand_livre_V2["categorie"][i] = grand_livre_V2["plan_compta"][i][0:1]
    
# vérification
grand_livre_V2 

### Création de la colonne "solde" , dérivée des colonnes existantes

In [ ]:
# on remplace les cellules vides de débit et de crédit par des 0
# affichage d'erreurs possible, ne pas en tenir compte
grand_livre_V2['debit'] = grand_livre_V2["debit"].fillna(0)

grand_livre_V2['credit'] = grand_livre_V2["credit"].fillna(0)

In [ ]:
grand_livre_V2["solde"] = grand_livre_V2["debit"] -  grand_livre_V2["credit"]
grand_livre_V2


### Mise en page

#### Création des colonnes mois / annee

In [ ]:
# mise en forme de la date en format date
grand_livre_V2["date"] = pd.to_datetime(grand_livre_V2["date"], format='%d/%m/%y')

grand_livre_V2["mois"] = grand_livre_V2["date"]
grand_livre_V2["mois"] = grand_livre_V2["date"].dt.month
grand_livre_V2["annee"] = grand_livre_V2["date"]
grand_livre_V2["annee"] = grand_livre_V2["date"].dt.year



#### Modifier l'ordre des colonnes

In [ ]:
grand_livre_V2 = grand_livre_V2[['annee','mois','date','categorie','plan_compta','Intitulé du compte général','debit','credit','solde','solde progressif']]
# vérification
grand_livre_V2


### Création d'un nouveau fichier excel 


In [ ]:
# création du fichier
# indiquer entre guillemets le nom du fichier qu'on veut créer
file_name = 'QR_rendu_2019-2020.xlsx'
grand_livre_V2.to_excel(file_name)
print('Sales record successfully exported into Excel File')


### Concaténation de deux dataframes

In [ ]:
rendu_2019_2020 = pd.read_excel("QR_rendu_2019-2020.xlsx")
rendu_2020_2021 = pd.read_excel("QR_rendu_2020-2021.xlsx")


In [ ]:
# vérifier au préalable que les deux tableaux à concaténer ont les mêmes intitulés aux colonnes
rendu_2019_2021 = pd.concat([rendu_2019_2020, rendu_2020_2021], axis=0)


In [ ]:
# supprimer la colonne unnamed 0
rendu_2019_2021 = rendu_2019_2021.drop(columns=["Unnamed: 0"])
# vérification
rendu_2019_2021


#### Rendu création excel

In [ ]:
# création du fichier
# indiquer entre guillemets le nom du fichier qu'on veut créer
file_name = 'QR_rendu_2019-2021.xlsx'
rendu_2019_2021.to_excel(file_name)
print('Sales record successfully exported into Excel File')


### Outils qui pourraient servir

#### exemple de recherche par mois et année

In [ ]:
mars_2020 = rendu_2019_2021.loc[(rendu_2019_2021["date"] >= "2020-03-01") & (rendu_2019_2021["date"] <= "2020-03-31")]


#### tracer un graphe

In [ ]:
nom_fichier = pd.read_excel("nom_fichier.xlsx")


In [ ]:
nom_fichier


In [ ]:
plt_av = nom_fichier[nom_fichier["nom_colonne"] == "attribut_colonne"]

In [ ]:
# affichage
plt_av.plot.scatter(x="dates", y="solde progressif", alpha=0.5)